In [ ]:
#default_exp utils

In [ ]:
#exporti 

from hypothesis import given, strategies as st
import pandas as pd
import inspect

In [ ]:
#hide
from nbdev.showdoc import show_doc

# Utilities

## Delegation
> Read more about it [in this brilliant post by Jeremy Howard](https://www.fast.ai/2019/08/06/delegation/).  
> We will use it to provide the functionality of pd.DataFrame to our BaseFrame Model

A delegate is a person sent or authorized to represent others. To delegate means to entrust a task or responsibility to another person.  

The `delegates` decorator takes the named arguments from one function and injects them into the signature of the function that is being decorated. This allows for auto-completion in a jupyter notebook and add doc string templates.

In [ ]:
#export

def delegates(to=None, keep=False):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    
    def _f(f):
        if to is None: 
            to_f,from_f = f.__base__.__init__,f.__init__
        else:          
            to_f,from_f = to,f
        
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd}
        sigd.update(s2)
        if keep: 
            sigd['kwargs'] = k
        from_f.__doc__ = f"""
        doc string from my delegate: `{to_f.__name__}`
        
        
        {to_f.__doc__}
        """
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

In [ ]:
import datetime as dt

def function_with_many_named_args(
    arg_1:str=None,
    arg_2:int=None,
    arg_3:dt.date=None
):
    """a description doc string"""
    return arg_1,arg_2,arg_3

show_doc(function_with_many_named_args)

In [ ]:
@delegates(function_with_many_named_args)
def just_one_more(extra,**kwargs):

    return list(kwargs.values())

show_doc(just_one_more)

## Hypothesis Testing

In [ ]:
# export

def dummy_df(Model):
    instances = []
    @given(st.builds(Model))
    def test_property(instance):
        instances.append(instance.dict())
    test_property()
    return pd.DataFrame.from_records(instances)

In [ ]:
from pydantic_pandas.core import BaseModel

class Record(BaseModel):
    string: str
    number: int

In [ ]:
dummy_df(Record).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   string  100 non-null    object
 1   number  100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [ ]:
#hide
!nbdev_build_lib

Converted 00_core.ipynb.
Converted 01_row_model.ipynb.
Converted 98_utils.ipynb.
Converted 99_default_standard_lib.ipynb.
Converted index.ipynb.
